In [6]:
# Importación de bibliotecas necesarias
import numpy as np
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from bitarray.util import ba2int

image_path = "/Users/carlosedm10/projects/college/TDSC/Files/i1.png"

In [7]:
# Función para transformar un entero e a un entero positivo e'
def transform_to_positive(e):
    return 2 * e if e >= 0 else 2 * abs(e) - 1


# Codificación Rice-m para un entero e
def rice_encode(e, m):
    e_prime = transform_to_positive(e)
    eq = e_prime // (2**m)  # Cociente
    er = e_prime % (2**m)  # Resto

    # Codificar eq con código unario
    c_eq = "0" * eq + "1"

    # Codificar er con longitud fija de m bits
    c_er = format(er, f"0{m}b")

    return c_eq + c_er


# Decodificación Rice-m para obtener el entero original e
def rice_decode(code, m):
    eq = code.find("1")  # Número de ceros antes del primer uno
    er = int(code[eq + 1 : eq + 1 + m], 2)  # Los m bits siguientes int (.., 2) bit to
    e_prime = eq * (2**m) + er

    # Recuperar e del e_prime
    if e_prime % 2 == 0:
        e = e_prime // 2
    else:
        e = -(e_prime + 1) // 2

    return e


# Función para aplicar el predictor de primera diferencia
def predictor_first_difference(x):
    x_pred = np.zeros_like(x)
    x_pred[1:] = x[:-1]
    return x_pred


# Función principal para codificar imágenes predictivamente
def cod_pred(image, output_path, m):
    # Cargar la imagen y convertir a escala de grises
    image = rgb2gray(image)
    x = np.asarray(image, dtype=np.int16)  # Convertir a array numpy

    # Dimensiones de la imagen
    rows, cols = x.shape

    # Transformar a una secuencia unidimensional (raster)
    x_raster = x.flatten()

    # Aplicar predictor de primera diferencia
    x_pred = predictor_first_difference(x_raster)
    e = x_raster - x_pred  # Error de predicción

    # Codificar los errores de predicción con Rice-m
    bitstream = ""
    for n, e_n in enumerate(e):
        if n == 0:
            # Codificar primera muestra con longitud fija de 8 bits
            bitstream += format(e_n, "08b")
        else:
            # Codificar con Rice-m
            bitstream += rice_encode(e_n, m)

    # Guardar los datos en un archivo binario
    with open(output_path, "wb") as f:
        # Escribir cabecera
        f.write(np.uint16(rows).tobytes())
        f.write(np.uint16(cols).tobytes())
        f.write(np.uint8(m).tobytes())

        # Escribir secuencia de bits
        f.write(int(bitstream, 2).to_bytes((len(bitstream) + 7) // 8, byteorder="big"))

    # Calcular tasa binaria
    nBits = len(bitstream)
    R = nBits / (rows * cols)
    return nBits, R


# Función para encontrar el valor óptimo de m
def find_optimal_m(image, m_values: [int]):
    best_m = None
    best_rate = float("inf")

    for m in m_values:
        _, R = cod_pred(image, f"encoded_m{m}.bin", m)
        print(f"m = {m}, Tasa binaria: {R:.2f} bits/pixel")
        if R < best_rate:
            best_rate = R
            best_m = m

    return best_m, best_rate

# Rice

In [8]:
coded_rice = rice_encode(-17, 4)
decoded_rice = rice_decode(coded_rice, 4)
print(f"Rice codificado: {coded_rice}, Rice decodificado: {decoded_rice}")

Rice codificado: 0010001, Rice decodificado: -17


# Codificador predictivo

In [9]:
m = 4
image = plt.imread(image_path)
nBits, R = cod_pred(image, "encoded_test_2.bin", m)
print(f"Tasa binaria: {R:.2f} bits/pixel")
decoded_rice = decode_pred("encoded_test_2.bin", m)

ARRAY [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
ARRAY FLATTEN [0.49836043 0.48184785 0.48380432 ... 0.25921726 0.2771435  0.24997531]
Tasa binaria: 5.00 bits/pixel


NameError: name 'decode_pred' is not defined

In [ ]:
m_values = [1, 2, 3, 4, 5]

image = plt.imread(image_path)
# Encontrar m óptimo
best_m, best_rate = find_optimal_m(image, m_values)
print(f"M óptimo: {best_m}, Tasa binaria mínima: {best_rate:.2f} bits/pixel")

decoded_image = decode_pred("")

In [ ]:
import numpy as np
from bitarray import bitarray
from bitarray.util import int2ba, ba2int


def decode2(encoded, m):
    # Example decoding process
    eq = encoded.find("1")  # Position of the first 1 (starts at 0)
    er_bits = encoded[eq + 1 : eq + 1 + m]  # Next m bits

    eq = eq
    er = ba2int(er_bits)

    ep = eq * (2**m) + er

    if ep % 2 == 0:
        e = ep // 2
    else:
        e = -(ep + 1) // 2

    return e

In [ ]:
_, R = cod_pred2(image_path, "encoded_m3.bin", 3)
print(f"Tasa binaria: {R:.2f} bits/pixel")

# Decodificar la imagen
decod_image = decode2("encoded_m3.bin", 3)